## HLCM estimation

Sam Maurer and Paul Waddell, August 2015

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os; os.chdir('..')
import models
import orca

In [ ]:
# Set up network aggregations and fitted prices

In [3]:
%%capture
orca.run(["neighborhood_vars"])

In [4]:
%%capture
orca.run(["rsh_simulate", "rrh_simulate", "nrh_simulate"])

## HLCM for owners

In [ ]:
# The model expression is in hlcm_owner.yaml and it's registered in models.py

In [7]:
orca.run(["hlcm_owner_estimate"])

Running step 'hlcm_owner_estimate'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -69003.870
Log-liklihood at convergence: -66749.138
Log-liklihood Ratio: 0.033

+-----------------------------+-------------+------------+---------+
| Component                   | Coefficient | Std. Error | T-Score |
+-----------------------------+-------------+------------+---------+
| np.log1p(residential_price) |    0.009    |   0.004    |  2.547  |
| np.log1p(residential_units) |    -0.112   |   0.006    | -17.638 |
| ave_income                  |    -0.465   |   0.010    | -44.424 |
| np.log1p(white * pct_white) |    6.161    |   0.009    | 705.067 |
| np.log1p(black * pct_black) |    6.192    |   0.011    | 555.209 |
| np.log1p(hisp * pct_hisp)   |    5.817    |   0.007    | 850.933 |
| np.log1p(asian * pct_asian) |    5.253    |   0.010    | 506.489 |
| autoPeakTotal               |    -0.139   |   0.010    | -14.355 |
+-----------------------------+-------------+------------+---------+
LCM RESULT

In [ ]:
# to try variations, create a new yaml file and run this to register it

@orca.model()
def hlcm_owner_estimate_NEW(households, residential_units, unit_aggregations):
    return utils.lcm_estimate("hlcm_owner_NEW.yaml", households, "unit_id",
                              residential_units, unit_aggregations)

orca.run(["hlcm_owner_estimate_NEW"])

## HLCM for renters

In [6]:
orca.run(["hlcm_renter_estimate"])

Running step 'hlcm_renter_estimate'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -62708.602
Log-liklihood at convergence: -58770.704
Log-liklihood Ratio: 0.063

+----------------------------------------------+-------------+------------+---------+
| Component                                    | Coefficient | Std. Error | T-Score |
+----------------------------------------------+-------------+------------+---------+
| np.log1p(unit_annual_rent / positive_income) |    0.001    |   0.003    |  0.224  |
| np.log1p(residential_units)                  |    0.054    |   0.006    |  9.084  |
| np.log1p(pct_renters)                        |    -0.101   |   0.066    |  -1.534 |
| ave_income                                   |    -1.230   |   0.024    | -50.220 |
| np.log1p(white * pct_white)                  |    3.000    |   0.005    | 571.576 |
| np.log1p(black * pct_black)                  |    3.000    |   0.007    | 418.667 |
| np.log1p(hisp * pct_hisp)                    |    3.000    | 

## Below is scratch work to check the status of underlying data

In [14]:
df1 = sim.get_table('residential_units').to_frame(
            columns=['unit_residential_price', 'node_id'])
df2 = sim.get_table('nodes').to_frame(
            columns=['sum_residential_units', 'ave_income', 'autoPeakTotal'])
df3 = sim.get_table('logsums').to_frame(
            columns=['autoPeakTotal', 'node_id'])

In [17]:
print(df1.describe())
print(df2.describe())
print(df3.describe())

       unit_residential_price
count          2785789.000000
mean               467.790277
std                912.510016
min                  0.000000
25%                330.009162
50%                397.432986
75%                491.675611
max             141696.511604
       sum_residential_units     ave_income
count          226060.000000  226060.000000
mean                5.959514      10.934405
std                 1.829774       1.963425
min                 0.000000       0.000000
25%                 5.517122      11.037583
50%                 6.443716      11.259089
75%                 7.027581      11.488537
max                10.149053      13.171155
       autoPeakTotal
count    1454.000000
mean       11.806047
std         0.814177
min         6.264300
25%        11.482000
50%        12.041550
75%        12.313925
max        13.082800


In [ ]:
# there's not much variation in any of those three aggregation terms, maybe try others?

In [18]:
df = sim.get_table('nodes').to_frame()

In [19]:
print(df.describe())

       sum_residential_units  sum_nonresidential_units  ave_sqft_per_unit  \
count          226060.000000             226060.000000      226060.000000   
mean                5.959514                  0.356974           7.253990   
std                 1.829774                  1.369450           1.322188   
min                 0.000000                  0.000000           0.000000   
25%                 5.517122                  0.000000           7.274734   
50%                 6.443716                  0.000000           7.436141   
75%                 7.027581                  0.000000           7.619552   
max                10.149053                  9.289473          12.261587   

       ave_lot_size_per_unit     population           poor        renters  \
count          226060.000000  226060.000000  226060.000000  226060.000000   
mean                8.929325       6.825222       5.364583       4.052799   
std                 1.933885       1.974722       1.966804       1.519042  

In [6]:
df = sim.get_table('households').to_frame()

In [7]:
df[:5]

,taz,serialno,puma5,income,persons,hht,unittype,noc,bldgsz,tenure,...,hadwpst,hadkids,bucketbin,originalpuma,hmultiunit,unit_id,building_id,income_quartile,zone_id,node_id
household_id,,,,,,,,,,,,,,,,,,,,,
227454,878,9279874,2404,17000,2,3,0,1,5,3,...,0,0,2,2404,1,-1,-1,1,-1,-1
2687058,1092,8097339,2103,0,1,0,2,0,0,0,...,0,0,9,2103,1,-1,-1,1,-1,-1
2687063,1092,3530666,2103,0,1,0,2,0,0,0,...,0,0,4,2103,1,-1,-1,1,-1,-1
2214782,930,5691854,2402,159700,3,5,0,0,2,1,...,0,0,4,2402,0,-1,-1,4,-1,-1
2687065,1092,8097339,2103,0,1,0,2,0,0,0,...,0,0,9,2103,1,-1,-1,1,-1,-1


In [15]:
len(df)

2732722

In [4]:
for a in list(df.columns.values):
    print "%s, %d" % (a, len(df[df[a].isnull()]))

taz, 0
serialno, 0
puma5, 0
income, 0
persons, 0
hht, 0
unittype, 0
noc, 0
bldgsz, 0
tenure, 0
vehicl, 0
hinccat1, 0
hinccat2, 0
hhagecat, 0
hsizecat, 0
hfamily, 0
hunittype, 0
hnoccat, 0
hwrkrcat, 0
h0004, 0
h0511, 0
h1215, 0
h1617, 0
h1824, 0
h2534, 0
h3549, 0
h5064, 0
h6579, 0
h80up, 0
hworkers, 0
hwork_f, 0
hwork_p, 0
huniv, 0
hnwork, 0
hretire, 0
hpresch, 0
hschpred, 0
hschdriv, 0
htypdwel, 0
hownrent, 0
hadnwst, 0
hadwpst, 0
hadkids, 0
bucketbin, 0
originalpuma, 0
hmultiunit, 0
building_id, 0
node_id, 0
income_quartile, 0
zone_id, 0


In [2]:
df = sim.get_table('buildings').to_frame()

Filling column building_type_id with value 1 (0 values)
Filling column residential_units with value 0 (0 values)
Filling column year_built with value 1966.0 (0 values)
Filling column non_residential_sqft with value 0 (0 values)
Filling column shape_area with value 604.336158108 (0 values)
Filling column zone_id with value 1178 (0 values)
Initial unit tenure assignment: 56% owner occupied, 4% unfilled


In [3]:
df[:5]

,parcel_id,residential_units,residential_sqft,non_residential_sqft,building_sqft,stories,year_built,redfin_sale_year,building_type_id,residential_price,...,zone_id,residential_rent,sqft_per_job,lot_size_per_unit,vacant_job_spaces,vacant_residential_units,general_type,sqft_per_unit,job_spaces,node_id
building_id,,,,,,,,,,,,,,,,,,,,,
1,69516,1,794,0,794,1,1939,2012,1,0,...,893,0,400,5200.062320,0,0,Residential,794,0,11584
2,69517,1,1600,0,1600,1,1939,2012,1,0,...,893,0,400,5200.063760,0,0,Residential,1600,0,11584
3,69518,1,1334,0,1334,1,1939,2012,1,0,...,893,0,400,5200.029703,0,0,Residential,1334,0,11585
4,69519,1,1220,0,1220,1,1939,2012,1,0,...,893,0,400,5200.144623,0,1,Residential,1220,0,11585
5,69520,1,892,0,892,1,1943,2012,1,0,...,893,0,400,5200.027559,0,0,Residential,892,0,11585


In [4]:
len(df)

1841288

In [5]:
for a in list(df.columns.values):
    print "%s, %d" % (a, len(df[df[a].isnull()]))

parcel_id, 0
residential_units, 0
residential_sqft, 0
non_residential_sqft, 0
building_sqft, 0
stories, 0
year_built, 0
redfin_sale_year, 0
building_type_id, 0
residential_price, 0
non_residential_price, 0
zone_id, 0
residential_rent, 0
sqft_per_job, 0
lot_size_per_unit, 0
vacant_job_spaces, 0
vacant_residential_units, 0
general_type, 0
sqft_per_unit, 0
job_spaces, 0
node_id, 0


In [5]:
sim.get_table('residential_units').to_frame()[:5]

Initial unit tenure assignment: 56% owner occupied, 4% unfilled


,building_id,deed_restricted,num_units,unit_num,unit_residential_price,unit_residential_rent,unit_tenure,vacant_units,submarket_id
unit_id,,,,,,,,,
0,1,0,1,0,0,0,1,0,893
1,2,0,1,0,0,0,0,0,893
2,3,0,1,0,0,0,0,0,893
3,4,0,1,0,0,0,0,1,893
4,5,0,1,0,0,0,0,0,893


In [4]:
sim.get_table('logsums').to_frame()[:5]

,autoPeakRetail,autoPeakTotal,autoOffPeakRetail,autoOffPeakTotal,transitPeakRetail,transitPeakTotal,transitOffPeakRetail,transitOffPeakTotal,nonMotorizedRetail,nonMotorizedTotal
taz,,,,,,,,,,
1,10.5416,13.0616,10.5312,13.0511,8.3954,11.1182,8.3003,11.0119,8.6157,11.3610
2,10.5336,13.0557,10.5185,13.0410,8.0848,10.8372,7.9369,10.6877,8.6104,11.3717
3,10.5082,13.0308,10.4998,13.0225,8.0456,10.7516,7.8563,10.5597,8.4067,11.1668
4,10.5609,13.0801,10.5458,13.0651,8.4306,11.1391,8.3426,11.0507,8.7110,11.4524
5,10.5458,13.0613,10.5325,13.0484,8.3395,11.0483,8.2755,10.9790,8.4145,11.1588


In [16]:
import numpy as np
print np.random.random()

0.555963700134
